#Python_task2

##Question 1 : Distance Matrix Calculation


In [ ]:
import pandas as pd

def calculate_distance_matrix(csv_file):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    # Create a dictionary to store distances between IDs
    distance_dict = {}

    # Iterate through the rows of the DataFrame
    for index, row in df.iterrows():
        id_start = row['id_start']
        id_end = row['id_end']
        distance = row['distance']

        # Add distances to the dictionary
        if (id_start, id_end) in distance_dict:
            distance_dict[(id_start, id_end)] += distance
        else:
            distance_dict[(id_start, id_end)] = distance

        # Account for bidirectional distances
        if (id_end, id_start) in distance_dict:
            distance_dict[(id_end, id_start)] += distance
        else:
            distance_dict[(id_end, id_start)] = distance

    # Create a list of unique IDs
    unique_ids = sorted(set(df['id_start'].unique()) | set(df['id_end'].unique()))

    # Create an empty DataFrame to store the distance matrix
    distance_matrix = pd.DataFrame(index=unique_ids, columns=unique_ids)

    # Recursive function to calculate distance between two points considering all possible paths
    def calculate_distance_recursive(id_i, id_j, visited):
        if id_i == id_j:
            return 0
        visited.add(id_i)
        possible_distances = []
        for neighbor_id in unique_ids:
            if neighbor_id not in visited:
                path1 = distance_dict.get((id_i, neighbor_id), None)
                if path1 is not None:
                    path2 = calculate_distance_recursive(neighbor_id, id_j, visited.copy())
                    if path2 is not None:
                        possible_distances.append(path1 + path2)
        visited.remove(id_i)
        return min(possible_distances) if possible_distances else None

    # Fill in the distance matrix using the recursive function
    for id_i in unique_ids:
        for id_j in unique_ids:
            distance_matrix.at[id_i, id_j] = calculate_distance_recursive(id_i, id_j, set())

    # Convert distance matrix to numeric values
    distance_matrix = distance_matrix.apply(pd.to_numeric, errors='coerce')

    return distance_matrix

# Example usage:
dataset_file = "dataset-3.csv"
distance_matrix = calculate_distance_matrix(dataset_file)
print(distance_matrix)


##Question 2: Unroll Distance Matrix

In [ ]:
def unroll_distance_matrix(distance_matrix):
    # Create an empty list to store DataFrames for each pair
    unrolled_dfs = []

    # Iterate through the rows of the distance matrix
    for id_start in distance_matrix.index:
        for id_end in distance_matrix.columns:
            # Skip same id_start to id_end combinations
            if id_start == id_end:
                continue

            # Get the distance value from the distance matrix
            distance = distance_matrix.at[id_start, id_end]

            # Create a DataFrame for the current pair
            current_df = pd.DataFrame({'id_start': [id_start], 'id_end': [id_end], 'distance': [distance]})

            # Append the current DataFrame to the list
            unrolled_dfs.append(current_df)

    # Concatenate all DataFrames in the list into a single DataFrame
    unrolled_df = pd.concat(unrolled_dfs, ignore_index=True)

    return unrolled_df

# Example usage:
unrolled_distance_df = unroll_distance_matrix(distance_matrix)
print(unrolled_distance_df)


##Question 3: Finding IDs within Percentage Threshold

In [ ]:
def find_ids_within_ten_percentage_threshold(input_df, reference_value):
    # Filter rows based on the reference_value
    reference_rows = input_df[input_df['id_start'] == reference_value]

    # Check if the reference_value exists in the DataFrame
    if reference_rows.empty:
        print(f"Reference value {reference_value} not found in the DataFrame.")
        return []

    # Calculate the average distance for the reference_value
    reference_average_distance = reference_rows['distance'].mean()

    # Calculate the threshold range (within 10%)
    lower_threshold = reference_average_distance * 0.9
    upper_threshold = reference_average_distance * 1.1

    # Filter rows based on the threshold range
    filtered_rows = input_df[(input_df['id_start'] != reference_value) &
                             (input_df['distance'] >= lower_threshold) &
                             (input_df['distance'] <= upper_threshold)]

    # Get the unique values from the id_start column and sort them
    result_ids = sorted(filtered_rows['id_start'].unique())

    return result_ids

# Example usage:

reference_value = 1001400
result_ids = find_ids_within_ten_percentage_threshold(unrolled_distance_df, reference_value)
print(result_ids)


##Question 4: Calculate Toll Rate

In [ ]:
def calculate_toll_rate(input_df):
    # Create a copy of the input DataFrame to avoid modifying the original
    df_with_toll = input_df.copy()

    # Define rate coefficients for each vehicle type
    rate_coefficients = {'moto': 0.8, 'car': 1.2, 'rv': 1.5, 'bus': 2.2, 'truck': 3.6}

    # Calculate toll rates for each vehicle type
    for vehicle_type, rate_coefficient in rate_coefficients.items():
        df_with_toll[vehicle_type] = df_with_toll['distance'] * rate_coefficient

    return df_with_toll

# Example usage:
df_with_toll = calculate_toll_rate(unrolled_distance_df)
print(df_with_toll)


##Question 5: Calculate Time-Based Toll Rates

In [ ]:
import numpy as np
from datetime import datetime as dt
from itertools import product

def calculate_time_based_toll_rates(result4):

    # Define the time ranges
    weekday_time_ranges = [
        (time(0, 0, 0), time(10, 0, 0)),
        (time(10, 0, 0), time(18, 0, 0)),
        (time(18, 0, 0), time(23, 59, 59))
    ]

    # Weekend time ranges
    weekend_time_ranges = [
        (time(0, 0, 0), time(23, 59, 59))
    ]

    # Create a DataFrame with the weekday time ranges
    weekday_time_ranges_df = pd.DataFrame(weekday_time_ranges, columns=['start_time', 'end_time'])

    weekday_time_ranges_df = pd.concat([weekday_time_ranges_df] * len(result4), ignore_index=True)

    # Add 'id' column to the weekday_time_ranges_df
    weekday_time_ranges_df['id_start'] = result4['id_start']
    weekday_time_ranges_df['id_end'] = result4['id_end']

    # Create a DataFrame with the weekend time ranges
    weekend_time_ranges_df = pd.DataFrame(weekend_time_ranges, columns=['start_time', 'end_time'])

    # Repeat the weekend time ranges for each unique id
    weekend_time_ranges_df = pd.concat([weekend_time_ranges_df] * len(result4), ignore_index=True)

    # Add 'id' column to the weekend_time_ranges_df
    weekend_time_ranges_df['id_start'] = result4['id_start']
    weekend_time_ranges_df['id_end'] = result4['id_end']


    result_df = pd.merge(result4, weekday_time_ranges_df, on=['id_start','id_end'])

    from itertools import product

    # Define the days of the week
    weekdays= ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday','Sunday']
    weekends=['Saturday','Sunday']
    # Generate all combinations of start and end days
    day_combinations_weekdays = list(product(weekdays, repeat=2))
    day_combinations_weekends = list(product(weekends, repeat=2))

    # Create a DataFrame with the day combinations
    day_combinations_df_weekdays = pd.DataFrame(day_combinations_weekdays, columns=['start_day', 'end_day'])
    day_combinations_df_weekends = pd.DataFrame(day_combinations_weekends, columns=['start_day', 'end_day'])

    # Repeat the day combinations for each unique id
    day_combinations_df_weekdays = pd.concat([day_combinations_df_weekdays] * len(result4), ignore_index=True)
    day_combinations_df_weekdays['id_start'] = result4['id_start']
    day_combinations_df_weekdays['id_end'] = result4['id_end']

    day_combinations_df_weekends = pd.concat([day_combinations_df_weekends] * len(result4), ignore_index=True)
    day_combinations_df_weekends['id_start'] = result4['id_start']
    day_combinations_df_weekends['id_end'] = result4['id_end']

    result_df=pd.merge(result_df,day_combinations_df_weekdays,on=['id_start','id_end'])
    weekend_comb=pd.merge(weekend_time_ranges_df,day_combinations_df_weekends,on=['id_start','id_end'])
    result_df1=pd.merge(result_df[['id_start','id_end','distance','moto','car','rv','bus','truck']],weekend_comb,on=['id_start','id_end'])
    final_df=pd.concat([result_df,result_df1],axis=0)

    #result_df = pd.merge(result_df, day_combinations_df, on=['id_start','id_end'])
    final_df['start_time']=np.where((final_df['start_day'].isin(['Saturday','Sunday']))&(final_df['end_day'].isin(['Saturday','Sunday'])),time(0,0,0),final_df['start_time'])
    final_df['end_time']=np.where((final_df['start_day'].isin(['Saturday','Sunday']))&(final_df['end_day'].isin(['Saturday','Sunday'])),time(23,59,59),final_df['end_time'])

    # Print the resulting DataFrame
    final_df=final_df.drop_duplicates()
    def coeff(x):
        if x[0] in ['Saturday','Sunday']:
            return 0.7
        else:
            if x[1]==time(0,0,0):
                return 0.8
            elif x[1]==time(10,0,0):
                return 1.2
            else:
                return 0.8
    final_df['coeff']=final_df[['start_day','start_time']].apply(coeff,axis=1)
    for col in ['moto','car','rv','bus','truck']:
        final_df[col]=final_df[col]*final_df['coeff']
    del final_df['coeff']
    return final_df